In [12]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library sqlite3

In [13]:
db_con = sqlite3.connect('../data/checking-logs.sqlite.sqlite')

## using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows

In [14]:
query = '''
--count average
SELECT case when test.first_commit_ts < test.first_view_ts
        THEN 'before' else 'after'
        end as time,
        AVG(CAST((julianday(test.first_commit_ts) -
      	julianday(datetime(deadlines.deadlines, 'unixepoch')))
     	* 24 AS integer)) AS avg_diff
FROM test
JOIN deadlines ON test.labname=deadlines.labs
WHERE labname<>'project1'
--users with after AND before values
AND uid in
(SELECT uid from
 --before,after commit
(SELECT test.uid,
        case when test.first_commit_ts < test.first_view_ts
        THEN 'before' else 'after'
        end as time
FROM test
JOIN deadlines ON test.labname=deadlines.labs
WHERE labname<>'project1')
GROUP by uid
HAVING COUNT(DISTINCT time)=2)
GROUP by time
'''
test_results = pd.io.sql.read_sql(query, db_con)
test_results

,time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [15]:
query = '''
--count average
SELECT case when control.first_commit_ts < control.first_view_ts
        THEN 'before' else 'after'
        end as time,
        AVG(CAST((julianday(control.first_commit_ts) -
      	julianday(datetime(deadlines.deadlines, 'unixepoch')))
     	* 24 AS integer)) AS avg_diff
FROM control
JOIN deadlines ON control.labname=deadlines.labs
WHERE labname<>'project1'
--users with after AND before values
AND uid in
(SELECT uid from
 --before,after commit
(SELECT control.uid,
        case when control.first_commit_ts < control.first_view_ts
        THEN 'before' else 'after'
        end as time
FROM control
JOIN deadlines ON control.labname=deadlines.labs
WHERE labname<>'project1')
GROUP by uid
HAVING COUNT(DISTINCT time)=2)
GROUP by time
'''
control_results = pd.io.sql.read_sql(query, db_con)
control_results

,time,avg_diff
0,after,-217.525
1,before,-95.500


## Close the connection

In [16]:
db_con.close()

## Did the hypothesis turn out to be true and the page does affect the students’ behavior?

Hypothesis is wrong ;(